# Rome Imperium Surrectum unit cross-checker
This notebook is designed to do two things:
1. create a csv file contains all the RIS units with their stats, descriptions and building requirements
2. check for inconsistencies between the export_descr_unit.txt and export_descr_buildings.csv

It was written for RIS 0.6.3, and uses regular expressions to process the text files, so any major changes 
to file structure that occur as a result of further development will probably break it.

Assumes data structures as per https://www.twcenter.net/forums/showthread.php?111344-The-Complete-EDU-Guide

## Creating unit csv

In [9]:
from ris_unit import ris_unit
import pandas as pd
import re

### Unit descriptions

In [10]:
# Get the unit descriptions first (as then we can add them as we go when we process the unit stats)
with open("input/export_units.txt", "r", encoding="utf-16") as file:

    # Ignore the first few lines until we get to the first unit
    line = file.readline()
    while line[0] != "{":
        line = file.readline()

    brace_count = 0
    unit_short_desc = ""
    unit_long_desc = ""
    # Store the unit descriptions in dictionaries with the key being the unit_dictionary_tag
    unit_names = {}
    unit_short_descs = {}
    unit_long_descs = {}

    while line:
        # New fields are signified with a brace, so that effectively acts as our delimiter"
        if line[0] == "{":
            brace_count += 1

        # We expect 3 braces per unit so if we get to brace count of 4 then we've hit the next unit and can store the previous one
        if brace_count == 4:
            unit_names[unit_dictionary_tag] = unit_name
            unit_short_descs[unit_dictionary_tag] = unit_short_desc.split("}")[1].replace("\\n", "\n").replace("\n\n\n","\n\n").strip()
            unit_long_descs[unit_dictionary_tag] = unit_long_desc.split("}")[1].replace("\\n", "\n").replace("\n\n\n","\n\n").strip()
            unit_short_desc = ""
            unit_long_desc = ""
            brace_count = 1

        # The first set of braces contains the unit type and name
        if brace_count == 1:
            unit_dictionary_tag = line.split("}")[0][1:]
            unit_name = line.split("}")[1]

        # The second set of braces contains the unit short description
        if brace_count == 2:
            unit_short_desc += line

        # The third set of braces contains the unit long description
        if brace_count == 3:
            unit_long_desc += line

        line = file.readline()
    else:
        # Handle the last unit (which won't get to brace count of 4)
        unit_names[unit_dictionary_tag] = unit_name
        unit_short_descs[unit_dictionary_tag] = unit_short_desc.split("}")[1].replace("\\n", "\n").replace("\n\n\n","\n\n").strip()
        unit_long_descs[unit_dictionary_tag] = unit_long_desc.split("}")[1].replace("\\n", "\n").replace("\n\n\n","\n\n").strip()

### Main unit file

In [11]:
# Process the export_descr_unit.txt file
# The core assumption here is that units are separated by lines that begin:"; COMMENTS"

units_stored = {}

with open("input/export_descr_unit.txt", "r") as file:
    line = file.readline()
    while line[0:10] != "; COMMENTS":
        line = file.readline()
    unit = ""

    while line:
        line = file.readline()
        if line[0:10] == "; COMMENTS":
            unit_data = ris_unit(unit)
            unit_data.add_name(unit_names[unit_data.dictionary])
            unit_data.add_short_desc(unit_short_descs[unit_data.dictionary])
            unit_data.add_long_desc(unit_long_descs[unit_data.dictionary])
            units_stored[unit_data.type] = unit_data
            unit = ""
        unit = unit + line

# Convert the units dictionary to a dataframe
units_df = []
for unit in units_stored.values():
    units_df.append(unit.to_series())

units_df = pd.concat(units_df, axis=1).T
units_df.to_csv("output/ris_units_no_buildings.csv", index=False)
units_df.head()

,type,category,unit_class,hp,attack,sec_attack,charge,def_skill,armour,shield,...,weapon_upgrade_cost,armour_upgrade_cost,custom_battle_cost,aor,ownership,region,dictionary,name,short_desc,long_desc
0,ballistas,siege,missile,1,6,25,0,7,6,0,...,624,13,353,False,"acarnania, achaea, acragas, aetolia, anatolian...",,ballistas,Ballistas,A Ballista is a sinew-powered weapon that look...,A Ballista is a sinew-powered weapon that look...
1,lithoboloi,siege,missile,1,6,30,0,7,6,0,...,827,17,511,False,"acarnania, achaea, acragas, aetolia, anatolian...",,lithoboloi,Lithoboloi,An enormous form of ballista that can hurl sto...,"In the ancient world, artillery was a weapon o..."
2,large_lithoboloi,siege,missile,1,6,35,0,7,6,0,...,1234,17,625,False,"acarnania, achaea, acragas, aetolia, anatolian...",,large_lithoboloi,Large Lithoboloi,An enormous form of ballista that can hurl sto...,"In the ancient world, artillery was a weapon o..."
3,barb peasant slave,infantry,light,1,6,0,1,5,0,0,...,1,120,3951,False,"acarnania, achaea, acragas, aedui, aetolia, al...",,barb_peasant_slave,Mob,A mob of people angry at the incompetence of t...,A mob of people angry at the incompetence of t...
4,roman velites,infantry,missile,1,10,6,2,10,3,5,...,0,46,388,False,"romans_julii, slave",,roman_velites,Velites,Velites were swift moving skirmish infantry ar...,"In the pre-Marian Republican legion, the Velit..."


### Buildings file

In [12]:
# Get the buildings
all_buildings = []
port_levels = {"port": 1, "shipwright": 2, "dockyard":3}
recruitment_building_pattern = re.compile("^\s+([a-z]+_recruitment[1-4]|port|shipwright|dockyard) requires")
recruit_unit_pattern = re.compile("^\s+recruit ")
current_building = "none"
current_faction = "none"
building_level = 0
unit_list = []
with open("input/export_descr_buildings.txt", "r") as file:
    line = file.readline()
    while line:
        if recruitment_building_pattern.match(line):
            if len(unit_list) > 0:
                unit_list = pd.concat(unit_list, axis=1).T
                unit_list["recruitment_building"] = current_building
                unit_list["faction"] = current_faction
                unit_list["building_level"] = building_level
                all_buildings.append(unit_list)
            # Get recruitment building (or port_building)
            try:
                current_building = re.findall("[0-4]", line.strip().split()[0])[0]
                building_level = re.search("[0-9]", line).group()
            except:
                # These are the port buildings
                current_building = line.split()[0]
                building_level = port_levels[current_building]
            if current_building == "dockyard":
                current_faction = re.findall("{([a-z, _]+)}", line)[0].strip()
            else:    
                current_faction = re.findall("{ ([a-z]+),[a-z1-9_ ,]*}", line)[0]
            #print(current_faction, building_level)
            unit_list = []
        elif recruit_unit_pattern.match(line):
            unit_name = re.findall("recruit \"([^{.]+)\"", line)[0]
            try:
                building = re.findall("} and ([a-z0-9_]+) ", line)[0]
                if building == "building_present_min_level":
                    building = re.findall("building_present_min_level port_buildings ([a-z0-9_]+)", line)[0]
                if building == "not":
                    building = ""
            except:
                building = ""
                #print(line)

            try:
                hidden_resources_not = re.findall("and not hidden_resource ([a-z0-9_]+)", line)
                if len(hidden_resources_not) == 0:
                    hidden_resources_not = ""
            except:
                hidden_resources_not = ""
                print(line)

            try:
                hidden_resources = re.findall("and hidden_resource ([a-z0-9_]+)", line)
                if len(hidden_resources) == 0:
                    hidden_resources = ""
            except:
                hidden_resources = ""
                print(line)

            try:
                event = re.findall("and major_event ([a-z0-9_]+)", line)
                if len(event) == 0:
                    event = ""
            except:
                event = ""
                print(line)

            try:
                not_player = re.findall("and not is_player", line)[0]
            except:
                not_player = ""

            # Treat naval buildings as both recruitment buildings and required buildings 
            if current_building in ["port", "shipwright", "dockyard"] and building == "":
                building = current_building
                
            # Exclude any lines that are not applicable to the player. If trying to debug AI armies then change the if condition
            if not_player == "":
                unit = pd.Series({"type": unit_name, "building": building, "hidden_resources":hidden_resources,
                                  "hidden_resources_not":hidden_resources_not, "event": event, "not_player":not_player})
                unit_list.append(unit)

        line = file.readline()
    # Ensure we store the last building's data
    else:
        if len(unit_list) > 0:
            unit_list = pd.concat(unit_list, axis=1).T
            unit_list["recruitment_building"] = current_building
            unit_list["faction"] = current_faction
            unit_list["building_level"] = building_level
            all_buildings.append(unit_list)

# Create a dataframe
all_buildings = pd.concat(all_buildings)

# Units are recorded against recruitment buildings of multiple levels. Keep only the lowest level of recruitment building. 
all_buildings.sort_values(["type", "faction", "building_level"], inplace=True)
all_buildings.drop_duplicates(["type", "faction"], keep="first", inplace=True)

# That doesn't quite work for dockyards since only specific factions can build a dockyard but all factions can build a shipyard
# So drop those manually. This will probably not work if someone adds naval units (but I guess that is unlikely)
all_buildings = all_buildings[~((all_buildings["recruitment_building"] == "dockyard") & ~(all_buildings["type"] == "naval quinquiremes"))]

all_buildings["hidden_resources"] = all_buildings["hidden_resources"].str.join(", ")
all_buildings["hidden_resources_not"] = all_buildings["hidden_resources_not"].str.join(", ")

all_buildings.head()

,type,building,hidden_resources,hidden_resources_not,event,not_player,recruitment_building,faction,building_level
104,achaian epilektoi,barracks_3,,,,,3,achaea,3
9,achaian epilektoi phalangites,barracks_4,,,,,4,achaea,4
44,achaian hoplites,barracks_2,achaian,,,,2,achaea,2
45,achaian peltast,barracks_2,achaian,,,,2,achaea,2
108,achaian peltophoroi,barracks_3,,,,,3,achaea,3


In [13]:
building_requirements = all_buildings.groupby(["type", "building", "recruitment_building", "hidden_resources", "hidden_resources_not", "event"]).agg({"faction": ", ".join}).reset_index()
building_requirements["different_rows"] = building_requirements.groupby("type")["faction"].transform("count")

In [14]:
all_buildings[all_buildings["type"] == "aor brittonic spearmen"]

,type,building,hidden_resources,hidden_resources_not,event,not_player,recruitment_building,faction,building_level
10,aor brittonic spearmen,barracks_1,brittonic,,,,1,acarnania,1
12,aor brittonic spearmen,barracks_1,brittonic,,,,1,achaea,1
10,aor brittonic spearmen,barracks_1,brittonic,,,,1,acragas,1
12,aor brittonic spearmen,barracks_1,brittonic,,,,1,aedui,1
10,aor brittonic spearmen,barracks_1,brittonic,,,,1,aetolia,1
...,...,...,...,...,...,...,...,...,...
12,aor brittonic spearmen,barracks_1,brittonic,,,,1,thracians,1
10,aor brittonic spearmen,barracks_1,brittonic,,,,1,trapezus,1
11,aor brittonic spearmen,barracks_1,brittonic,,,,1,triballi,1
12,aor brittonic spearmen,barracks_1,brittonic,,,,1,tylis,1


### Combine units and buildings

In [5]:
# Combine the unit data with building requirements

# Flag where there are multiple rows for the same unit, but then only keep the first
building_requirements = all_buildings.groupby(["type", "building", "recruitment_building", "hidden_resources", "hidden_resources_not", "event"]).agg({"faction": ", ".join}).reset_index()
building_requirements["different_rows"] = building_requirements.groupby("type")["faction"].transform("count")
building_requirements.drop_duplicates(subset="type", keep="first", inplace=True)

# Combine the unit data with building requirements
units_df_combined = units_df.merge(building_requirements, on="type", how="left")
units_df_combined.to_csv("output/ris_units_combined.csv", index=False)
units_df_combined.head()

,type,category,unit_class,hp,attack,sec_attack,charge,def_skill,armour,shield,...,name,short_desc,long_desc,building,recruitment_building,hidden_resources,hidden_resources_not,event,faction,different_rows
0,ballistas,siege,missile,1,6,25,0,7,6,0,...,Ballistas,A Ballista is a sinew-powered weapon that look...,A Ballista is a sinew-powered weapon that look...,siege_engineer_1,3,,,,"acarnania, achaea, acragas, aetolia, anatolian...",1.0
1,lithoboloi,siege,missile,1,6,30,0,7,6,0,...,Lithoboloi,An enormous form of ballista that can hurl sto...,"In the ancient world, artillery was a weapon o...",siege_engineer_1,3,,,,sparta,2.0
2,large_lithoboloi,siege,missile,1,6,35,0,7,6,0,...,Large Lithoboloi,An enormous form of ballista that can hurl sto...,"In the ancient world, artillery was a weapon o...",siege_engineer_2,4,,,,"acarnania, achaea, acragas, aetolia, anatolian...",1.0
3,barb peasant slave,infantry,light,1,6,0,1,5,0,0,...,Mob,A mob of people angry at the incompetence of t...,A mob of people angry at the incompetence of t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,roman velites,infantry,missile,1,10,6,2,10,3,5,...,Velites,Velites were swift moving skirmish infantry ar...,"In the pre-Marian Republican legion, the Velit...",missiles_1,1,,,,roman,1.0


## Checking for issues / inconsistencies

### Check for differeing building requirements across factions

In [6]:
# These are the frequencies of building requirements
all_buildings["building"].value_counts()

building
barracks_2          5588
missiles_1          4684
barracks_1          3996
equestrian_2        2895
missiles_2          2538
barracks_3          2379
equestrian_3        2020
missiles_3          1343
equestrian_1        1165
port                1157
equestrian_4         902
barracks_4           365
siege_engineer_2     108
siege_engineer_1      69
                       3
barracks_3_roman       3
shipwright             2
dockyard               1
barracks_4_roman       1
Name: count, dtype: int64

In [7]:
# Look at any with blank building requirements
all_buildings[all_buildings["building"] == ""].to_csv("output/blank_building_requirements.csv", index=False)
all_buildings[all_buildings["building"] == ""]

,type,building,hidden_resources,hidden_resources_not,event,not_player,recruitment_building,faction,building_level
301,aspidophoroi1,,,,,,3,athens,3
298,aspidophoroi2,,,,,,3,selge,3
103,machimoi swordsmen,,,ai_cant_recruit,,,1,ptolemaic,1


In [8]:
# Identify cases where building requirements are inconsistent between factions
building_requirements1 = all_buildings.groupby(["type", "building"]).count()
building_requirements1 = building_requirements1.reset_index().groupby("type").count()
inconsistent_names1 = building_requirements1[building_requirements1["building"]> 1].index.to_list()
inconsistent_buildings1 = all_buildings[all_buildings["type"].isin(inconsistent_names1)]
inconsistent_buildings1.groupby(["type", "building", "recruitment_building"]).agg({"faction": ", ".join}).reset_index()

,type,building,recruitment_building,faction
0,asian slingers,missiles_1,1,"cilicians, paphlagonia, pontus"
1,asian slingers,missiles_2,2,"armenia, atropatene, cappadocia, parni"
2,aspidophoroi1,,3,athens
3,aspidophoroi1,equestrian_3,3,"achaea, acragas, anatolians, argos, boeotia, e..."
4,aspidophoroi2,,3,selge
5,aspidophoroi2,equestrian_3,3,"byzantium, chios, cius, epirus, greeks, miletu..."
6,greek hoplites,barracks_1,1,"acragas, argos, byzantium, chios, cius, epirus..."
7,greek hoplites,barracks_2,2,anatolians
8,greek peltasts,barracks_2,2,"cyzicus, olbia"
9,greek peltasts,missiles_2,2,"acarnania, anatolians, argos, athens, bactria,..."


In [9]:
# Identify cases where hidden resources are inconsistent between factions
building_requirements2 = all_buildings.groupby(["type", "hidden_resources", "hidden_resources_not"])["faction"].count()
building_requirements2 = building_requirements2.reset_index().groupby("type").count()
inconsistent_names2 = building_requirements2[building_requirements2["faction"]>1].index.to_list()
inconsistent_buildings2 = all_buildings[all_buildings["type"].isin(inconsistent_names2)]
inconsistent_buildings2 = inconsistent_buildings2.groupby(["type", "hidden_resources", "hidden_resources_not"]).agg({"faction": ", ".join}).reset_index()
inconsistent_buildings2

,type,hidden_resources,hidden_resources_not,faction
0,aor greek hoplites,greek,"prienian, olbian, milesian, kyzikan, knossian,...","acarnania, achaea, aedui, aetolia, allobroges,..."
1,aor greek hoplites,greek,"prienian, olbian, milesian, kyzikan, knossian,...",sparta
2,aor greek peltasts,greek,"herakleiote, sicel, milesian, aitolian, achaian","achaea, acragas, aedui, aetolia, allobroges, a..."
3,aor greek peltasts,greek,"herakleiote, sicel, milesian, aitolian, achaia...",sparta
4,argive epilektoi,,,argos
...,...,...,...,...
59,thureophoroi cavalry,greek,,"antigonid, bosporan, epirus, lysiad, ptolemaic..."
60,thureophoroi cavalry,massaliote,,massalia
61,thureophoroi cavalry,syracusan,,syracuse
62,towered forest elephants,,,"carthage, egypt, masaesyli, massylii"


In [10]:
# Identify cases where building requirements or hidden resources are inconsistent between factions (i.e. either of the above, but also including recruitment building level)
building_requirements3 = all_buildings.groupby(["type", "building", "recruitment_building", "hidden_resources", "hidden_resources_not", "event"]).agg({"faction": ", ".join}).reset_index()
inconsistent_buildings3 = building_requirements3.groupby("type").count()
inconsistent_buildings3_names = inconsistent_buildings3[inconsistent_buildings3["faction"] > 1].index.to_list()
inconsistent_buildings3 = building_requirements3[building_requirements3["type"].isin(inconsistent_buildings3_names)]
inconsistent_buildings3.to_csv("output/inconsistent_buildings.csv", index=False)
inconsistent_buildings3

,type,building,recruitment_building,hidden_resources,hidden_resources_not,event,faction
48,aor ankonitan archers,missiles_1,1,,,,"acarnania, achaea, acragas, aedui, aetolia, al..."
49,aor ankonitan archers,missiles_1,4,,,,greeks
50,aor ankonitan hoplites,barracks_1,1,,,,"acarnania, achaea, acragas, aedui, aetolia, al..."
51,aor ankonitan hoplites,barracks_1,4,,,,greeks
76,aor athamanian peltasts,missiles_2,2,,,,"acarnania, achaea, acragas, aedui, aetolia, al..."
...,...,...,...,...,...,...,...
869,thureophoroi cavalry,equestrian_2,2,massaliote,,,massalia
870,thureophoroi cavalry,equestrian_2,2,syracusan,,,syracuse
874,towered forest elephants,equestrian_3,3,,,,"masaesyli, massylii"
875,towered forest elephants,equestrian_3,3,libyan,,,"massalia, ptolemaic"


### Check AoR

In [11]:
# Units typically have up to three flavours - base unit, aor unit and merc unit.
# Check that the three flavours are consistent across the unit and buildings file. 
# It is possible that the merc units might not be recruitable by buildings, but otherwise things should be consistent
df_cross_check = units_df[["type"]].copy()
building_cross_check = building_requirements[["type"]].drop_duplicates()

units_cross_check = pd.concat([df_cross_check, building_cross_check], axis=0)

# Create the aor and merc unit names
units_cross_check["aor_name"] = units_cross_check["type"]
units_cross_check.loc[~(units_cross_check["type"].str.startswith("aor")), "aor_name"] = "aor " + units_cross_check.loc[~(units_cross_check["type"].str.startswith("aor")), "type"]
units_cross_check["merc_name"] = units_cross_check["type"]
units_cross_check.loc[~(units_cross_check["type"].str.startswith("merc")), "merc_name"] = "merc " + units_cross_check.loc[~(units_cross_check["type"].str.startswith("merc")), "type"]

# Reset any that start with aor or merc and then drop duplicates
units_cross_check.loc[(units_cross_check["type"].str.startswith("aor")), "type"] = units_cross_check.loc[(units_cross_check["type"].str.startswith("aor")), "type"].str[4:]
units_cross_check.loc[(units_cross_check["type"].str.startswith("merc")), "type"] = units_cross_check.loc[(units_cross_check["type"].str.startswith("merc")), "type"].str[4:]
units_cross_check.drop_duplicates(subset="type", inplace=True)

# Check the existing of all of the unit variations in both files
units_cross_check = units_cross_check.merge(df_cross_check, how="left", on="type", indicator="in_units")
units_cross_check = units_cross_check.merge(df_cross_check, left_on="aor_name", right_on="type", suffixes=["","aor_units"], how="left", indicator="aor_units")
units_cross_check = units_cross_check.merge(df_cross_check, left_on="merc_name", right_on="type", suffixes=["","merc_units"], how="left", indicator="merc_units")
units_cross_check = units_cross_check.merge(building_cross_check, how="left", on="type", indicator="in_buildings")
units_cross_check = units_cross_check.merge(building_cross_check, left_on="aor_name", right_on="type", suffixes=["","aor_buildings"], how="left", indicator="aor_buildings")
units_cross_check = units_cross_check.merge(building_cross_check, left_on="merc_name", right_on="type", suffixes=["","merc_buildings"], how="left", indicator="merc_buildings")
units_cross_check = units_cross_check[["type", "in_units", "aor_units", "merc_units", "in_buildings", "aor_buildings", "merc_buildings"]]
units_cross_check.replace({"left_only": 0}, inplace=True)
units_cross_check.replace({"both": 1}, inplace=True)
units_cross_check.set_index("type", inplace=True)

In [12]:
# Filter to only units that have issues:
# 1. the base unit exists in one file but not the other
# 2. the aor unit exists in one file but not the other
# 3. the merc unit exists in the buildings file but not the units file
units_cross_check_issues = units_cross_check.copy()
units_cross_check_issues = units_cross_check_issues[((units_cross_check_issues["in_units"]) != (units_cross_check_issues["in_buildings"]))
                                                    | (units_cross_check_issues["aor_units"] != units_cross_check_issues["aor_buildings"])
                                                    | ((units_cross_check_issues["merc_units"] == "False") & (units_cross_check_issues["merc_buildings"] == "True"))
                                                    ]
units_cross_check_issues.to_csv("output/unit_cross_check_issues.csv")
units_cross_check_issues

,in_units,aor_units,merc_units,in_buildings,aor_buildings,merc_buildings
type,,,,,,
barb peasant slave,1,0,0,0,0,0
italic velites,1,1,0,0,1,0
italic hastati,1,1,0,0,1,0
italic principes,1,1,0,0,0,0
italic triarii,1,1,0,0,0,0
...,...,...,...,...,...,...
legio xxii primigenia,1,0,0,0,0,0
legio xxii primigenia first,1,0,0,0,0,0
roman early praetorian,1,0,0,0,0,0


In [13]:
# Check that aor units all have the same stats as the equivalent base units
aor_check = units_df.copy()
aor_check["aor_type"] = "aor " + aor_check["type"]
aor_units = units_df[units_df["aor"]]
base_units = aor_check.drop(columns="type").merge(aor_units["type"], left_on="aor_type", right_on="type", how="inner")
base_units.set_index("type", inplace=True)
aor_units.set_index("type", inplace=True)

aor_units = aor_units.apply(pd.to_numeric, errors="ignore")
aor_units = aor_units.select_dtypes(include="number")
base_units = base_units.apply(pd.to_numeric, errors="ignore")
base_units = base_units.select_dtypes(include="number")

aor_unit_diffs = aor_units - base_units
# Some units exist as only aor and have NAs, so ignore these
aor_unit_diffs = aor_unit_diffs.dropna()

# Show any cases where the aor unit has a different stat to the base unit
aor_unit_diffs = aor_unit_diffs[aor_unit_diffs.any(axis=1) != 0]
aor_unit_diffs.to_csv("output/aor_unit_diffs.csv", index=False)
aor_unit_diffs

,hp,attack,sec_attack,charge,def_skill,armour,shield,missile_range,missile_ammo,recruit_cost,...,scrub_mdf,sand_mdf,forest_mdf,snow_mdf,morale,charge_distance,recruit_turns,weapon_upgrade_cost,armour_upgrade_cost,custom_battle_cost
type,,,,,,,,,,,,,,,,,,,,,


In [ ]:
# aor nubian cavalry - why barracks_2?